# CS246 - Colab 9
## Studying COVID-19

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [9]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u292-b10-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


Now we authenticate a Google Drive client to download the files we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [10]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [11]:
id='1YT7ttUAafCjbVdm6obeHp1TWAK0rEtoR'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_covid19_confirmed_global.csv')

id='1YxEA5UQ2EFJ_9oLssM__Gs1ncVNufGNA'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_covid19_deaths_global.csv')

id='1CNxszuZTeIw-5cF5yrzKMZdb1qV0hSoy'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_covid19_recovered_global.csv')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import IntegerType

Let's initialize the Spark context.

In [15]:
sc = SparkContext.getOrCreate()
sc.stop()

# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [ ]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### Data Loading

In this Colab, we will be analyzing the timeseries data of the Coronavirus COVID-19 Global Cases, collected by Johns Hopkins CSSE.

Here you can check a realtime dashboard based on this dataset: [https://www.arcgis.com/apps/opsdashboard/index.html?fbclid=IwAR2hQKsEZ3D38wVtXGryUhP9CG0Z6MYbUM_boPEaV8FBe71wUvDPc65ZG78#/bda7594740fd40299423467b48e9ecf6](https://www.arcgis.com/apps/opsdashboard/index.html?fbclid=IwAR2hQKsEZ3D38wVtXGryUhP9CG0Z6MYbUM_boPEaV8FBe71wUvDPc65ZG78#/bda7594740fd40299423467b48e9ecf6)

---



*   ```confirmed```: dataframe containing the cumulative number of confirmed COVID-19 cases, divided by geographical area
*   ```deaths```: dataframe containing the cumulative number of deaths due to COVID-19, divided by geographical area
*   ```recovered```: dataframe containing the cumulative number of recoevered patients, divided by geographical area

The data sets contain data entries for each day, representing the cumulative totals as of that day.







In [16]:
confirmed = spark.read.csv('time_series_covid19_confirmed_global.csv', header=True)
deaths = spark.read.csv('time_series_covid19_deaths_global.csv', header=True)
recovered = spark.read.csv('time_series_covid19_recovered_global.csv', header=True)

### Your Task

We are aware of the stress we are all experiencing because of the still-ongoing pandemic and the fact that many of you have projects and exams due this week due to the lack of a finals week. As such, we decided to conclude our series of Colabs with a **lightweight task** -- given everything you have learned about Spark during the quarter, this Colab should take you just a few minutes to complete.

Consider the entries for May 1, 2021, in the timeseries, and compute:


*   number of confirmed COVID-19 cases across the globe
*   number of deaths due to COVID-19 (across the globe)
*   number of recovered patients across the globe



In [22]:
print(confirmed.select(sum("5/1/21")).collect())
print(deaths.select(sum("5/1/21")).collect())
print(recovered.select(sum("5/1/21")).collect())

[Row(sum(5/1/21)=152196159.0)]
[Row(sum(5/1/21)=3192930.0)]
[Row(sum(5/1/21)=88919401.0)]


Consider the data points for March 1, 2020, and March 1, 2021, and filter out the geographical locations where less than 50 cases have been confirmed.
For the areas still taken into consideration after the filtering step, compute the ratio between number of deaths and number of confirmed cases.

In [53]:
df1 = confirmed.filter(col("3/1/20") >= 50)
df1 = df1.filter(col("Country/Region") == "China")
df1 = df1.select("Province/State", "3/1/20").collect()

In [57]:
df1_list = []
for i in range(len(df1)):
  df1_list.append((df1[i]["Province/State"], df1[i]["3/1/20"]))

In [65]:
ratio = 0
for numRow in range(len(df1_list)):
  name = df1_list[numRow][0]
  numCases = int(df1_list[numRow][1])
  d = deaths.filter(col("Province/State") == name)
  d = d.select("3/1/20")
  numDeaths = int(d.collect()[0]["3/1/20"])
  nextRatio = numDeaths / numCases
  if nextRatio > ratio:
    ratio = nextRatio
    nameMaxRatio = name

print(nameMaxRatio)

Hubei


Consider the data points for March 1, 2021, and May 1, 2021, in the timeseries, and filter out the geographical locations where less than 50 deaths have been confirmed (as of March 1, 2021).
For the areas still taken into consideration after the filtering step, compute the percent increase in cumulative deaths between the two dates.

In [178]:
df2 = confirmed.filter(col("3/1/21") >= 50)
df2 = df2.withColumn("3/1/21", df2["3/1/21"].cast(IntegerType()))
df2 = df2.groupBy("Country/Region").sum("3/1/21").collect()

In [126]:
df2_list = []
for i in range(len(df2)):
  df2_list.append((df2[i]["Country/Region"], df2[i]["sum(3/1/21)"]))

In [174]:
df2_d = deaths.withColumn("3/1/21", deaths["3/1/21"].cast(IntegerType()))
df2_d = df2_d.groupBy("Country/Region").sum("3/1/21")

In [176]:
ratio = 0
for numRow in range(len(df2_list)):
  name = df2_list[numRow][0]
  numCases = int(df2_list[numRow][1])
  d = df2_d.filter(col("Country/Region") == name)
  d = d.select("sum(3/1/21)")
  numDeaths = int(df2_d.filter(col("Country/Region") == name).collect()[0]["sum(3/1/21)"])
  nextRatio = numDeaths / numCases
  if nextRatio > ratio:
    ratio = nextRatio
    nameMaxRatio = name

print(nameMaxRatio)

Yemen


---

In [186]:
df3 = deaths.filter(col("3/1/21") >= 50)
df3 = df3.withColumn("3/1/21", df3["3/1/21"].cast(IntegerType()))
df3 = df3.withColumn("5/1/21", df3["5/1/21"].cast(IntegerType()))
df3 = df3.groupBy("Country/Region").sum("3/1/21", "5/1/21").collect()

In [196]:
df3_list = []
for i in range(len(df3)):
  df3_list.append((df3[i]["Country/Region"], df3[i]["sum(3/1/21)"], df3[i]["sum(5/1/21)"]))

In [200]:
percentage = 0
for numRow in range(len(df3_list)):
  name = df3_list[numRow][0]
  numCases_mar = int(df3_list[numRow][1])
  numCases_maj = int(df3_list[numRow][2])
  nextPercentage = ((numCases_maj - numCases_mar) / numCases_mar) * 100
  if nextPercentage > percentage:
    percentage = nextPercentage
    nameMaxPerc = name
print(nameMaxPerc)

Uruguay


Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!

---
---
# Results:
---
*   Q2.1 : 5/18/21
*   Q2.2 : 152196159
*   Q2.3 : 3192930
*   Q2.4 : 88919401
*   Q2.5 : Hubei
*   Q2.6 : Yemen
*   Q2.7 : Uruguay
---

